In [ ]:
pip install opencv-python

In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

### Task A

I defined the solution for the rotation transformation in the following way:

1. Get the dimension from the image's shape, then calculate the center of the image and set up the transformation matrix
2. Then I compute the new image's dimensions based on the transformation to accommodate the changes. This was based on a combination of trial and error and trying to make it as close to the transformed size of the image for the rotation case. For this reason I have included an if condition which checks if sin(theta) > 1/sqrt(2) to make sure we are always scaling up the overall window size for the new image.
3. Then I begin by create a list of the original coordinates for each pixel, which are relative to the center of the image, and then apply the transformation matrix to each pixel’s coordinate.
4. I then adjust the transformed coordinates back for the new image's dimensions and map them to the new image space by subtracting from the new center.
5. After this when putting the pixel values for the image I check the bounds so that the image doesn't clip and so that I don't get an out of bounds issue.

In [ ]:
def ICV_rotate_image_array(image, theta):
    # Step 1 (Initial Setup)

    width, height = image.shape # Get image shape
    theta = np.deg2rad(theta) # convert theta to radians
    # initialize rotation matrix
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta),  np.cos(theta)]
    ])

    # Step 2 (Computing new image dimensions)

    # Absolute value of sin(theta) and cos(theta)
    abssin, abscos = np.abs(np.sin(theta)), np.abs(np.cos(theta))
    # Makes sure that we are scaling the width and height of the transformed image appropriately
    if abssin >= np.sqrt(1/2):
        new_w, new_h = int(width * abssin + height * abssin), int(width * abssin + height * abssin)
    else:
        new_w, new_h = int(width * abscos + height * abscos), int(width * abscos + height * abscos)

    # create the image with these new dimensions
    new_image = np.zeros((new_w, new_h), dtype=image.dtype)

    # get the center of the original image
    center_x, center_y = width / 2, height / 2
    # get the center of the new image
    new_center_x, new_center_y = new_w / 2, new_h / 2

    # Step 3 (Create a matrix with original coordinates for the pixels in the image)

    coords = np.array([[[i - center_x, j - center_y] for j in range(height)] for i in range(width)])

    # Tranform these coordinates by multiplying the rotation matrix
    
    coords_transformed = np.array([[rotation_matrix @ coords[i][j] for j in range(height)] for i in range(width)])
    

    for i in range(width):
        for j in range(height):
            # Step 4 (Adjust to the coordinate system for the new image)
            new_x, new_y = int(coords_transformed[i][j][0] + new_center_x), int(coords_transformed[i][j][1] + new_center_y)

            # Step 5 (bound checking)
            if 0 <= new_x < new_w and 0 <= new_y < new_h:
                new_image[new_x, new_y] = image[i, j]
    
    return new_image


## Horizontal Skew Transformation

I follow a similar approach to the rotation task but slightly different. I divided this task in the following steps:

1. Get the dimension from the image's shape, then calculate the center of the image and set up the transformation matrix
2. Then I compute the new image's dimensions based on the transformation to accommodate the changes. In this case it is simpler than the rotation case as only the width of the image changes, which is easier to transform the frame for. 
3. I transform the coordinates of the pixels in the new image based on the horizontal skew transformation. Instead of using the matrix here I apply it directly in the final for loop to reduce complexity of the code.
4. After this when putting the pixel values for the image I check the bounds so that the image doesn't clip and so that I don't get an out of bounds issue. This time I only need to check the bounds along the x axis.

This task was simpler as the transformation can be applied without changing the center of the system, which significantly simplifying the problem.

In [ ]:
def ICV_skew_image_array(image, theta):
    # Step 1: Initial Setup
    height, width = image.shape  # Corrected order: (H, W) 

    theta = np.deg2rad(theta)  # Convert theta to radians

    # Step 2: Compute new image dimensions based on skew angle
    new_width = int(width + height * abs(np.tan(theta)))
    new_height = height  # Only width changes for horizontal skew

    # Create new image with calculated dimensions
    new_image = np.zeros((new_height, new_width), dtype=image.dtype)

    # Step 3: Transform coordinates for skew
    for y in range(height):
        for x in range(width):
            # The operations below is equivalent to applying a horizontal skew matrix 
            new_x = int(x + y * np.tan(theta)) 
            new_y = 0 + y 

            # Step 4: Bound checking before assigning pixel
            if 0 <= new_x < new_width:
                new_image[new_y, new_x] = image[y, x]

    return new_image

# Task B

In [ ]:
im = cv2.imread("Name_Image_Arial_72.png")
im_g = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
plt.imshow(im_g, cmap="gray")

In [ ]:
for theta in [30, 60, 120, -50]:
    plt.imshow(ICV_rotate_image_array(im_g, theta), cmap='gray')
    plt.savefig(f"Rotate im {theta}")
    plt.show()

In [ ]:
for theta in [10, 40, 60]:
    plt.imshow(ICV_skew_image_array(im_g, theta), cmap='gray')
    plt.savefig(f"Skew im {theta}")
    plt.show()

# Task C

In [ ]:
theta1, theta2 = -20, 50
rot_im =  ICV_rotate_image_array(im_g, theta1)
skew_im = ICV_skew_image_array(im_g, theta2)

In [ ]:
plt.imshow(rot_im, cmap="gray")
plt.show()
plt.imshow( ICV_skew_image_array(rot_im, theta1), cmap='gray')
plt.savefig(f"First Rotate by {theta1} clockwise and then Skew by {theta2}")
plt.show()

In [ ]:
plt.imshow(skew_im, cmap="gray")
plt.show()
plt.imshow(ICV_rotate_image_array(skew_im, theta1), cmap='gray')
plt.savefig(f"First Skew by {theta2} and then Rotate by {theta1}")
plt.show()